In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
CityMv3 = pd.read_csv('CityMaster1.2.csv', converters={'FIPS': str})
CityMv3.shape
# CityMv3.columns

(3898, 79)

In [3]:
CityMv3['FIPS_2digit'] = CityMv3['FIPS'].str[:2]

# # Calculate value counts for the first two digits
# value_counts = CityMv3['FIPS_2digit'].value_counts()

# # Loop through each unique pair of first two digits and print their counts
# for fips_prefix, count in value_counts.items():
#     print(f"FIPS Prefix: {fips_prefix}, Count: {count}")

In [4]:
# make contiguous US only
codes = pd.read_excel('FIPS.xlsx', converters={'Numeric code': str})

# make list of 'Numeric code' where 'Status' does not contain "State"
# in CityMv3, drop rows where 'FIPS_2digit' macthes any element from the above list

non_state_fips = codes.loc[codes['Status'] != "State", 'Numeric code'].tolist()

CityMv3_contiguous = CityMv3[~CityMv3['FIPS_2digit'].isin(non_state_fips)]

# drop Alaska (02) and Hawaii (15)
CityMv3_contiguous = CityMv3[~CityMv3['FIPS_2digit'].isin(['02', '15'])]

CityMv3.shape

(3898, 80)

In [5]:
# map county data to cities

# create City_Dummy where 0 if City_Pop is 0 or null and 1 otherwise
# for all null columns in rows with City_Dummy, map data from the overlying county (row with FIPS equal to the first 5 digits of the city row)

# create a sub df that only contains cities but keep the df with both cities and counties as well


CityMv3['City_Dummy'] = np.where((CityMv3['City_Pop'] == 0) | pd.isnull(CityMv3['City_Pop']), 0, 1)

In [6]:
# Iterate through each row
for index, row in CityMv3.iterrows():
    if row['City_Dummy'] == 1:  # It's a city row
        county_fips = row['FIPS'][:5]
        county_row = CityMv3[CityMv3['FIPS'] == county_fips]  # Get the corresponding county row
        # Now, for each column that is null in the city row, fill it with data from the county row
        for col in CityMv3.columns:
            if pd.isnull(row[col]):
                # Assuming there's exactly one county row matching the FIPS, so we use .iloc[0]
                CityMv3.at[index, col] = county_row[col].iloc[0] if not county_row.empty else row[col]

In [7]:
CityMv3_cities_only = CityMv3[CityMv3['City_Dummy'] == 1]

In [8]:
CityMv3_cities_only.shape

(93, 81)

In [9]:
CityMv3.to_csv('CityMaster1.3.csv')